# Simulation cMAB

This notebook shows a simulation framework for the contextual multi-armed bandit (cMAB). It allows to study the behaviour of the bandit algoritm, to evaluate results and to run experiments on simulated data under different context, reward and action settings.

In [ ]:
import pandas as pd
from sklearn.datasets import make_classification

from pybandits.core.cmab import Cmab
from pybandits.utils.simulation_cmab import SimulationCmab

First we need to define the simulation parameters: (i) the number of samples per batch to consider at each iteration of the simulation, (ii) the number of samples groups (we assume to have groups of samples whose features come from the same distribution), (iii) the numbe of updates in the simulation, (iv) the list of possible actions, (v) the number of features in the context matrix.

Data are processed in batches of size n>=1. Per each batch of simulated samples, the cMAB selects one action and collects the corresponding simulated reward for each sample. Then, prior parameters are updated based on returned rewards from recommended actions.

In [ ]:
# simulation parameters
batch_size = 100
n_groups = 3
n_updates = 5
n_jobs = 1
actions_ids = ["action A", "action B", "action C"]
n_features = 5
verbose = True

Here, we init the context matrix $X$ and the groups of samples. Samples that belong to the same group have features that come from the same distribution.

In [ ]:
# init context matrix and groups
X, group = make_classification(
    n_samples=batch_size * n_updates, n_features=n_features, n_informative=n_features, n_redundant=0, n_classes=n_groups
)

We need to define the probabilities of positive rewards per each action/group, i.e. the ground truth ('Action A': 0.8 for group '0' means that if the bandits selects 'Action A' for samples that belong to group '0', then the environment will return a positive reward with 80% probability).

In [ ]:
# init probability of rewards from the environment
prob_rewards = pd.DataFrame(
    [[0.05, 0.80, 0.05], [0.80, 0.05, 0.05], [0.80, 0.05, 0.80]], columns=actions_ids, index=range(n_groups)
)
print("Probability of positive reward for each group/action:")
prob_rewards

We initialize the Cmab as shown in the previous notebook and the SimulationCmab with the parameters set above.

In [ ]:
# init contextual Multi-Armed Bandit model
cmab = Cmab(n_features=n_features, actions_ids=actions_ids, n_jobs=n_jobs)

In [ ]:
# init simulation
sim = SimulationCmab(
    cmab=cmab, X=X, group=group, batch_size=batch_size, n_updates=n_updates, prob_rewards=prob_rewards, verbose=verbose
)

Now, we can start simulation process by executing run() which performs the following steps:
```
For i=0 to n_updates:
    Extract batch[i] of samples from X
    Model recommends the best actions as the action with the highest reward probability to each simulated sample in batch[i] and collect corresponding simulated rewards
    Model priors are updated using information from recommended actions and returned rewards
```
Finally, we can visualize the results of the simulation. As defined in the ground truth: 'Action B' was the action recommended the most for samples that belong to group '0', 'Action A' to group '1' and both 'Action A' and 'Action C' to group '2'.

In [ ]:
sim.run()